In [9]:
import numpy as np 
import urdf2casadi.urdfparser as u2c
from urdf2casadi.geometry import plucker
from urdf_parser_py.urdf import URDF, Pose
import PyKDL as kdl
import kdl_parser_py.urdf as kdlurdf
from timeit import Timer, timeit, repeat
import rbdl
import pybullet as pb

def median(lst):
    n = len(lst)
    if n < 1:
            return None
    if n % 2 == 1:
            return sorted(lst)[n//2]
    else:
            return sum(sorted(lst)[n//2-1:n//2+1])/2.0
        
def average(lst): 
    return sum(lst) / len(lst) 

#pbmodel = pb.loadURDF("pantilt.urdf")
sim = pb.connect(pb.DIRECT)
#pbmodel = pb.loadURDF("/urdf4timing/1dof.urdf")
ndofs = 10
urdf_nr = list(range(ndofs + 1))
urdf_nr.pop(0)


nitr = 1000
nrepeat = 100
    

path_to_urdf = "/home/lmjohann/urdf2casadi/examples/timing/urdf4timing/24dof.urdf"
root = 'base_link'
tip = "link16"

ok, kdl_tree = kdlurdf.treeFromFile(path_to_urdf)
kdlmodel = kdl_tree.getChain(root,tip)

rbdlmodel = rbdl.loadModel(path_to_urdf)
pbmodel = pb.loadURDF(path_to_urdf, useFixedBase=True, flags = pb.URDF_USE_INERTIA_FROM_FILE)
asd = u2c.URDFparser()
robot_desc = asd.from_file(path_to_urdf)

jlist, names, qmax, qmin = asd.get_joint_info(root, tip)
njoints = asd.get_n_joints(root, tip)

gravity_u2c = [0, 0, -9.81]


#u2c and pybullet declarations
q_none = [None]*njoints
qdot_none = [None]*njoints
qddot_none = [None]*njoints
g_pb = [None]*njoints


tau_none = [None]*njoints

g_u2c = asd.get_gravity_rnea(root, tip, gravity_u2c)
c_u2c = asd.get_coriolis_rnea(root, tip)
m_u2c = asd.get_inertia_matrix_crba(root, tip)
fd_aba_u2c = asd.get_forward_dynamics_aba(root, tip, gravity_u2c)
fd_crba_u2c = asd.get_forward_dynamics_crba(root, tip, gravity_u2c)
id_u2c = asd.get_inverse_dynamics_rnea(root, tip, gravity_u2c)




The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
/home/lmjohann/urdf2casadi/examples/timing/urdf4timing/24dof.urdf


In [10]:
import casadi as cs
qs =cs.SX.sym("qs",njoints)
qsdot =cs.SX.sym("qsdot",njoints)
qsddot =cs.SX.sym("qsddot",njoints)
qsdddot = cs.SX.sym("qsdddot", njoints)
taus =cs.SX.sym("taus",njoints)
tausdot =cs.SX.sym("tausdot",njoints)

c_vec = cs.vertcat(qs, qsdot)
c_dvec = cs.vertcat(qsdot, qsddot)
id_vec = cs.vertcat(qs, qsdot, qsddot)
id_dvec = cs.vertcat(qsdot, qsdot, qsdddot)
fd_vec = cs.vertcat(qs, qsdot, qsddot)
fd_dvec = cs.vertcat(qsdot, qsdot, tausdot)

dc = cs.jacobian(c_u2c(qs, qsdot), c_vec)
dc = cs.Function("dc", [qs, qsdot], [dc], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dctimes = cs.jtimes(c_u2c(qs, qsdot), c_vec, c_dvec)
dctimes = cs.Function("dctimes", [qs, qsdot, qsddot], [dctimes], {"jit": True, "jit_options":{"flags":"-Ofast"}})

did = cs.jacobian(id_u2c(qs, qsdot, qsddot), id_vec)
did = cs.Function("did", [qs, qsdot, qsddot], [did], {"jit": True, "jit_options":{"flags":"-Ofast"}})

didtimes = cs.jtimes(id_u2c(qs, qsdot, qsddot), id_vec, id_dvec)
didtimes = cs.Function("didtimes", [qs, qsdot, qsddot, qsdddot], [didtimes], {"jit": True, "jit_options":{"flags":"-Ofast"}})


dfd_aba = cs.jacobian(fd_aba_u2c(qs, qsdot, taus), fd_vec)
dfd_aba = cs.Function("dfd_aba", [qs, qsdot, taus], [dfd_aba], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dfdtimes_aba = cs.jtimes(fd_aba_u2c(qs, qsdot, taus), fd_vec, fd_dvec)
dfdtimes_aba = cs.Function("dfdtimes_aba", [qs, qsdot, taus, qsddot, tausdot], [dfdtimes_aba], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dfd_crba = cs.jacobian(fd_crba_u2c(qs, qsdot, taus), fd_vec)
dfd_crba = cs.Function("dfd_crba", [qs, qsdot, taus], [dfd_crba], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dfdtimes_crba = cs.jtimes(fd_crba_u2c(qs, qsdot, taus), fd_vec, fd_dvec)
dfdtimes_crba = cs.Function("dfdtimes_crba", [qs, qsdot, taus, qsddot, tausdot], [dfdtimes_crba], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dg = cs.jacobian(g_u2c(qs), qs)
dg = cs.Function("dg", [qs], [dg], {"jit": True, "jit_options":{"flags":"-Ofast"}})


dm = cs.jacobian(m_u2c(qs), qs)
dm = cs.Function("dm", [qs], [dm], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dgtimes = cs.jtimes(g_u2c(qs), qs, qsdot)
dgtimes = cs.Function("dgtimes", [qs, qsdot], [dgtimes], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dmtimes = cs.jtimes(m_u2c(qs), qs, qsdot)
dmtimes = cs.Function("dmtimes", [qs, qsdot], [dmtimes], {"jit": True, "jit_options":{"flags":"-Ofast"}})


print "\nur5_nodes_did = ", did.n_nodes()
print "\nur5_nodes_dc = ", dc.n_nodes()
print "\nur5_nodes_dm = ", dm.n_nodes()
print "\nur5_nodes_dg = ", dg.n_nodes()
print "\nur5_nodes_dfd_aba = ", dfd_aba.n_nodes()
print "\nur5_nodes_dfd_crba =", dfd_crba.n_nodes()

print "\nur5_nodes_didtimes = ", didtimes.n_nodes()
print "\nur5_nodes_dctimes = ", dctimes.n_nodes()
print "\nur5_nodes_dmtimes = ", dmtimes.n_nodes()
print "\nur5_nodes_dgtimes = ", dgtimes.n_nodes()
print "\nur5_nodes_dfdtimes_aba = ", dfdtimes_aba.n_nodes()
print "\nur5_nodes_dfdtimes_crba =", dfdtimes_crba.n_nodes()


def dfd_crba_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dfd_crba(q_none, qdot_none, tau_none)
   
def dfdtimes_crba_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        taudot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dfdtimes_crba(q_none, qdot_none, tau_none, qddot_none, taudot_none)

def dfd_aba_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dfd_aba(q_none, qdot_none, tau_none)
   
def dfdtimes_aba_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        taudot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dfdtimes_aba(q_none, qdot_none, tau_none, qddot_none, taudot_none)
   
   
   
def dm_func():  
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2  
    dm(q_none)
   
def dg_func():  
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dg(q_none)
   
   
def dmtimes_func():  
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2  
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dmtimes(q_none, qdot_none)
   
def dgtimes_func():  
    for j in range(njoints):
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2        
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dgtimes(q_none, qdot_none)


def dc_func():  
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    dc(q_none, qdot_none)
   
   
def dctimes_func():  
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    dctimes(q_none, qdot_none, qddot_none)
   
   
def did_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    did(q_none, qdot_none, qddot_none)
   
def didtimes_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    didtimes(q_none, qdot_none, qddot_none, qdddot_none)

qdddot_none = [None]*njoints
taudot_none = [None]*njoints





ur5_nodes_did =  43583

ur5_nodes_dc =  46363

ur5_nodes_dm =  62159

ur5_nodes_dg =  8240

ur5_nodes_dfd_aba =  135573

ur5_nodes_dfd_crba = 516309

ur5_nodes_didtimes =  6177

ur5_nodes_dctimes =  6011

ur5_nodes_dmtimes =  18357

ur5_nodes_dgtimes =  1860

ur5_nodes_dfdtimes_aba =  24030

ur5_nodes_dfdtimes_crba = 63676


In [ ]:
#nodes 32 dof
ur5_nodes_did =  43583

ur5_nodes_dc =  46363

ur5_nodes_dm =  62159

ur5_nodes_dg =  8240

ur5_nodes_dfd_aba =  135573

ur5_nodes_dfd_crba = 516309

ur5_nodes_didtimes =  6177

ur5_nodes_dctimes =  6011

ur5_nodes_dmtimes =  18357

ur5_nodes_dgtimes =  1860

ur5_nodes_dfdtimes_aba =  24030

ur5_nodes_dfdtimes_crba = 63676


In [7]:
timeit_did = repeat("did_func()", setup = "from __main__ import did_func", repeat = nrepeat, number = nitr)
mediantime_did = median(timeit_did)/nitr*1000000

timeit_dg = repeat("dg_func()", setup = "from __main__ import dg_func", repeat = nrepeat, number = nitr)
mediantime_dg = median(timeit_dg)/nitr*1000000

timeit_dc = repeat("dc_func()", setup = "from __main__ import dc_func", repeat = nrepeat, number = nitr)
mediantime_dc = median(timeit_dc)/nitr*1000000

timeit_dm = repeat("dm_func()", setup = "from __main__ import dm_func", repeat = nrepeat, number = nitr)
mediantime_dm = median(timeit_dm)/nitr*1000000

timeit_dfd_aba = repeat("dfd_aba_func()", setup = "from __main__ import dfd_aba_func", repeat = nrepeat, number = nitr)
mediantime_dfd_aba = median(timeit_dfd_aba)/nitr*1000000

timeit_dfd_crba = repeat("dfd_crba_func()", setup = "from __main__ import dfd_crba_func", repeat = nrepeat, number = nitr)
mediantime_dfd_crba = median(timeit_dfd_crba)/nitr*1000000

   
print "\nur5_did =", mediantime_did
print "\nur5_dg =", mediantime_dg
print "\nur5_dc =", mediantime_dc
print "\nur5_dm =", mediantime_dm
print "\nur5_dfd_aba =", mediantime_dfd_aba
print "\nur5_dfd_crba =", mediantime_dfd_crba


ur5_did = 87.4503850937

ur5_dg = 29.1750431061

ur5_dc = 77.1050453186

ur5_dm = 65.5740499496

ur5_dfd_aba = 126.462459564

ur5_dfd_crba = 328.211545944


In [ ]:
#32 dof
ur5_did = 87.4503850937

ur5_dg = 29.1750431061

ur5_dc = 77.1050453186

ur5_dm = 65.5740499496

ur5_dfd_aba = 126.462459564

ur5_dfd_crba = 328.211545944



#24 dof
ur5_did = 85.678935051

ur5_dg = 28.932094574

ur5_dc = 76.2020349503

ur5_dm = 65.5000209808

ur5_dfd_aba = 125.869870186

ur5_dfd_crba = 324.005961418

In [8]:
timeit_didtimes = repeat("didtimes_func()", setup = "from __main__ import didtimes_func", repeat = nrepeat, number = nitr)
mediantime_didtimes = median(timeit_didtimes)/nitr*1000000

timeit_dgtimes = repeat("dgtimes_func()", setup = "from __main__ import dgtimes_func", repeat = nrepeat, number = nitr)
mediantime_dgtimes = median(timeit_dgtimes)/nitr*1000000

timeit_dctimes = repeat("dctimes_func()", setup = "from __main__ import dctimes_func", repeat = nrepeat, number = nitr)
mediantime_dctimes = median(timeit_dctimes)/nitr*1000000

timeit_dmtimes = repeat("dmtimes_func()", setup = "from __main__ import dmtimes_func", repeat = nrepeat, number = nitr)
mediantime_dmtimes = median(timeit_dmtimes)/nitr*1000000

timeit_dfdtimes_aba = repeat("dfdtimes_aba_func()", setup = "from __main__ import dfdtimes_aba_func", repeat = nrepeat, number = nitr)
mediantime_dfdtimes_aba = median(timeit_dfdtimes_aba)/nitr*1000000

timeit_dfdtimes_crba = repeat("dfdtimes_crba_func()", setup = "from __main__ import dfdtimes_crba_func", repeat = nrepeat, number = nitr)
mediantime_dfdtimes_crba = median(timeit_dfdtimes_crba)/nitr*1000000

print "\nur5_didtimes =", mediantime_didtimes
print "\nur5_dgtimes =", mediantime_dgtimes
print "\nur5_dctimes =", mediantime_dctimes
print "\nur5_dmtimes =", mediantime_dmtimes
print "\nur5_dfdtimes_aba =", mediantime_dfdtimes_aba
print "\nur5_dfdtimes_crba =", mediantime_dfdtimes_crba


ur5_didtimes = 80.6254148483

ur5_dgtimes = 42.5009727478

ur5_dctimes = 63.6094808578

ur5_dmtimes = 62.0809793472

ur5_dfdtimes_aba = 94.326376915

ur5_dfdtimes_crba = 144.921064377


In [ ]:
#32 dof 
ur5_didtimes = 80.6254148483

ur5_dgtimes = 42.5009727478

ur5_dctimes = 63.6094808578

ur5_dmtimes = 62.0809793472

ur5_dfdtimes_aba = 94.326376915

ur5_dfdtimes_crba = 144.921064377



#24 dof
ur5_didtimes = 79.8690319061

ur5_dgtimes = 41.9964790344

ur5_dctimes = 62.9850625992

ur5_dmtimes = 61.6065263748

ur5_dfdtimes_aba = 92.9458141327

ur5_dfdtimes_crba = 144.107937813